In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mtp
import seaborn as sb
import plotly.express as px

In [ ]:
data_year = pd.read_csv("data_by_year.csv")
data = pd.read_csv('data.csv')

In [ ]:
data_year.info()

In [ ]:
data_year

In [ ]:
data_year.isnull().sum()

In [ ]:
data

In [ ]:
data.isnull().sum()

In [ ]:
features = ['acousticness','danceability', 'energy','instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo', 'duration_ms', 'key']

In [ ]:
numeric_song_data = data[features]

In [ ]:
from sklearn.preprocessing import StandardScaler
#scaling data
song_scaler = StandardScaler()
scaled_song_data = song_scaler.fit_transform(numeric_song_data)

#clusteriong the scaled data
from sklearn.cluster import KMeans
song_cluster = KMeans(n_clusters = 20)
song_cluster.fit(scaled_song_data)

In [ ]:
# labeling song clusters
song_cluster_labels = song_cluster.labels_

# addinng cluster lables to the datset
data['clusters'] = song_cluster_labels

# reducing the dimensionality for graph plotting.
# as data has linear relation instead of TSNE we use PCA

from sklearn.decomposition import PCA

song_reducer = PCA(n_components = 2)
reduced_song_data = song_reducer.fit_transform(scaled_song_data)

In [ ]:
new_song_dataset = pd.DataFrame(columns = ['x-axis', 'y-axis'], data = reduced_song_data)
new_song_dataset['year'] = data['year']
new_song_dataset['clusters'] = data['clusters']

In [ ]:
song_graph = px.scatter(new_song_dataset, x = 'x-axis', y = 'y-axis', color = 'clusters', hover_data = ['x-axis', 'y-axis', 'clusters'])
song_graph.show()

# Recommending songs.
    If the input is song name.

In [ ]:
def ArtistBasedRecommender1(song_name):
#     finding the cluster of the song
    song_cluster = data.loc[data['name'] == song_name, 'clusters'].values[0]
#     finding the year of the song
    song_year = data.loc[data['name'] == song_name, 'year'].values[0]
#     finding similar cluster songs
    similar_cluster_songs = data.loc[data['clusters'] == song_cluster]
#     finding the similar year songs from similar clustered songs
    similar_year_songs = similar_cluster_songs.loc[similar_cluster_songs['year'] == song_year]
    recommending_songs = similar_year_songs.loc[similar_year_songs['name'] != song_name]
    top_recommending_songs = recommending_songs.nlargest(5, 'popularity')
#     top_songs = top_recommending_songs['name']
#     empty list
    ls = []
    
    for index,row in top_recommending_songs.iterrows():
        name_of_song = row['name']
        ls.append(name_of_song)
    return ls

In [ ]:
best_songs = ArtistBasedRecommender1('Clancy Lowered the Boom')
for i in best_songs:
    print(i)

# Recommending songs
    If the input is year.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def ArtistBasedRecommender2(song_year):

    similar_year_songs = data.loc[data['year'] == song_year]
    
    top_recommending_songs = similar_year_songs.nlargest(5, 'popularity')
    
    for index,row in top_recommending_songs.iterrows():
        song_name = row['name']
        print(song_name)


In [ ]:
best_songs = ArtistBasedRecommender2(1921)
best_songs